In [43]:
from flask import Flask, request
import json
import psycopg2
import psycopg2.extras
import os
import time
import requests
import pandas as pd

In [21]:
api_host = os.getenv("API_HOST", "http://0.0.0.0:8080")
print(api_host)

http://0.0.0.0:8080


In [46]:
respuesta = requests.get(api_host)
data_raw  = respuesta.json()
df = pd.DataFrame.from_dict(pd.json_normalize(data_raw), orient="columns")
df 

,id,sepallengthcm,sepalwidthcm,petallengthcm,petalwidthcm,species
0,0,5.1,3.5,1.4,0.2,Iris-setosa
1,1,4.9,3.0,1.4,0.2,Iris-setosa
2,2,4.7,3.2,1.3,0.2,Iris-setosa
3,3,4.6,3.1,1.5,0.2,Iris-setosa
4,4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,145,6.7,3.0,5.2,2.3,Iris-virginica
146,146,6.3,2.5,5.0,1.9,Iris-virginica
147,147,6.5,3.0,5.2,2.0,Iris-virginica
148,148,6.2,3.4,5.4,2.3,Iris-virginica
